# NOTE: THIS PROJECT IS STILL IN PROGRESS.

# Predicting the Revenue of restaurants using Machine Learning

In this notebook, we will predicting the revenue of restaurants with a given dataset with NaN values and non-number columns. Time Series Regression

## Problem

How well can we predict the future revenue of a restaurant, given its characteristics previous examples of how much similar revenues a restaurant generates?

## Data

1. Id : Restaurant id. 
2. Open Date : opening date for a restaurant
3. City : City that the restaurant is in. Note that there are unicode in the names. 
4. City Group: Type of the city. Big cities, or Other. 
5. Type: Type of the restaurant. FC: Food Court, IL: Inline, DT: Drive Thru, MB: Mobile
6. P1, P2 - P37: There are three categories of these obfuscated data. Demographic data are gathered from third party providers with GIS systems. These include population in any given area, age and gender distribution, development scales. Real estate data mainly relate to the m2 of the location, front facade of the location, car park availability. Commercial data mainly include the existence of points of interest including schools, banks, other QSR operators.
7. Revenue: The revenue column indicates a (transformed) revenue of the restaurant in a given year and is the target of predictive analysis. Please note that the values are transformed so they don't mean real dollar values. 

## Evaluation metric

We will use the root mean squared log error (RMSLE) of the data trained by the RandomForestRegressor model for evaluation.

In [1]:
# Import data analysis tools 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn

In [2]:
# Import the training set
df = pd.read_csv("restaurant-data/train.csv",
                low_memory=False,
                 parse_dates=["Open Date"])

In [3]:
df.head().T

,0,1,2,3,4
Id,0,1,2,3,4
Open Date,1999-07-17 00:00:00,2008-02-14 00:00:00,2013-03-09 00:00:00,2012-02-02 00:00:00,2009-05-09 00:00:00
City,İstanbul,Ankara,Diyarbakır,Tokat,Gaziantep
City Group,Big Cities,Big Cities,Other,Other,Other
Type,IL,FC,IL,IL,IL
P1,4,4,2,6,3
P2,5,5,4,4.5,4
P3,4,4,2,6,3
P4,4,4,5,6,4
P5,2,1,2,4,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Id          137 non-null    int64         
 1   Open Date   137 non-null    datetime64[ns]
 2   City        137 non-null    object        
 3   City Group  137 non-null    object        
 4   Type        137 non-null    object        
 5   P1          137 non-null    int64         
 6   P2          137 non-null    float64       
 7   P3          137 non-null    float64       
 8   P4          137 non-null    float64       
 9   P5          137 non-null    int64         
 10  P6          137 non-null    int64         
 11  P7          137 non-null    int64         
 12  P8          137 non-null    int64         
 13  P9          137 non-null    int64         
 14  P10         137 non-null    int64         
 15  P11         137 non-null    int64         
 16  P12         137 non-null  

In [5]:
df.isna().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

In [8]:
# Sort dataframe by open date
df.sort_values(by=["Open Date"], inplace=True, ascending=True)
df['Open Date'].head(20)

35    1996-05-08
122   1997-04-10
28    1998-01-28
112   1998-03-14
64    1998-07-13
83    1998-08-12
54    1999-06-05
0     1999-07-17
27    1999-10-09
99    1999-12-18
16    2000-01-07
85    2000-01-07
92    2002-11-01
131   2002-11-06
102   2004-09-11
44    2004-10-13
121   2004-11-13
124   2005-08-18
75    2005-10-15
81    2005-12-16
Name: Open Date, dtype: datetime64[ns]

In [14]:
# Copy dataframe
df_tmp = df.copy()

In [15]:
# Add datetime parameters for saledate
df_tmp["reveuneYear"] = df_tmp['Open Date'].dt.year
df_tmp["revenueMonth"] = df_tmp['Open Date'].dt.month
df_tmp["revenueDay"] = df_tmp['Open Date'].dt.day
df_tmp["revenueDayofweek"] = df_tmp['Open Date'].dt.dayofweek
df_tmp["revenueDayofyear"] = df_tmp['Open Date'].dt.dayofyear

# Drop original saledate
df_tmp.drop("Open Date", axis=1, inplace=True)

In [16]:
df_tmp.head().T

,35,122,28,112,64
Id,35,122,28,112,64
City,Adana,İstanbul,İstanbul,İstanbul,İstanbul
City Group,Other,Big Cities,Big Cities,Big Cities,Big Cities
Type,IL,IL,IL,IL,IL
P1,4,12,4,4,12
P2,5,7.5,5,5,7.5
P3,4,6,4,5,6
P4,3,7.5,4,4,6
P5,1,2,1,1,2
P6,2,8,3,5,8


In [17]:
# Convert non numeric objects to categories
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [18]:
df_tmp.head(20)

,Id,City,City Group,Type,P1,P2,P3,P4,P5,P6,...,P34,P35,P36,P37,revenue,reveuneYear,revenueMonth,revenueDay,revenueDayofweek,revenueDayofyear
35,35,Adana,Other,IL,4,5.0,4.0,3.0,1,2,...,3,4,3,3,3903884.0,1996,5,8,2,129
122,122,İstanbul,Big Cities,IL,12,7.5,6.0,7.5,2,8,...,18,3,16,6,4286645.0,1997,4,10,3,100
28,28,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,1,3,...,3,5,4,4,4705945.0,1998,1,28,2,28
112,112,İstanbul,Big Cities,IL,4,5.0,5.0,4.0,1,5,...,2,1,3,3,3784230.0,1998,3,14,5,73
64,64,İstanbul,Big Cities,IL,12,7.5,6.0,6.0,2,8,...,24,15,20,6,2551252.0,1998,7,13,0,194
83,83,İstanbul,Big Cities,IL,2,4.0,4.0,4.0,2,5,...,4,4,5,3,5966193.0,1998,8,12,2,224
54,54,Muğla,Other,IL,2,4.0,5.0,4.0,1,1,...,0,0,0,0,4651866.0,1999,6,5,5,156
0,0,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,...,5,4,3,4,5653753.0,1999,7,17,5,198
27,27,Bursa,Other,FC,4,5.0,4.0,3.0,1,2,...,0,0,0,0,5435276.0,1999,10,9,5,282
99,99,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,4,...,5,5,5,3,13575224.0,1999,12,18,5,352


In [19]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 35 to 21
Data columns (total 47 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Id                137 non-null    int64   
 1   City              137 non-null    category
 2   City Group        137 non-null    category
 3   Type              137 non-null    category
 4   P1                137 non-null    int64   
 5   P2                137 non-null    float64 
 6   P3                137 non-null    float64 
 7   P4                137 non-null    float64 
 8   P5                137 non-null    int64   
 9   P6                137 non-null    int64   
 10  P7                137 non-null    int64   
 11  P8                137 non-null    int64   
 12  P9                137 non-null    int64   
 13  P10               137 non-null    int64   
 14  P11               137 non-null    int64   
 15  P12               137 non-null    int64   
 16  P13               137 non-

In [22]:
# Convert categorial objects to numbers
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_tmp[label] = pd.Categorical(content).codes+1 

In [24]:
df_tmp.head().T

,35,122,28,112,64
Id,35.0,122.0,28.0,112.0,64.0
City,1.0,32.0,32.0,32.0,32.0
City Group,2.0,1.0,1.0,1.0,1.0
Type,3.0,3.0,3.0,3.0,3.0
P1,4.0,12.0,4.0,4.0,12.0
P2,5.0,7.5,5.0,5.0,7.5
P3,4.0,6.0,4.0,5.0,6.0
P4,3.0,7.5,4.0,4.0,6.0
P5,1.0,2.0,1.0,1.0,2.0
P6,2.0,8.0,3.0,5.0,8.0


## Read test data

In [28]:
df_test = pd.read_csv("restaurant-data/test.csv",
                     low_memory=False,
                     parse_dates=["Open Date"])

In [29]:
df_test.head().T

,0,1,2,3,4
Id,0,1,2,3,4
Open Date,2011-01-22 00:00:00,2011-03-18 00:00:00,2013-10-30 00:00:00,2013-05-06 00:00:00,2013-07-31 00:00:00
City,Niğde,Konya,Ankara,Kocaeli,Afyonkarahisar
City Group,Other,Other,Big Cities,Other,Other
Type,FC,IL,FC,IL,FC
P1,1,3,3,2,2
P2,4,4,4,4,4
P3,4,4,4,4,4
P4,4,4,4,4,4
P5,1,2,2,2,1


In [30]:
df_test.isna().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
dtype: int64

In [31]:
# Sort dataframe by open date
df_test.sort_values(by=["Open Date"], inplace=True, ascending=True)
df_test['Open Date'].head(20)

16457   1995-05-08
93003   1995-05-08
70787   1995-05-08
50778   1995-05-08
32290   1995-05-08
69385   1995-05-08
27707   1995-05-08
23269   1995-05-08
52821   1995-05-08
32261   1995-05-08
36717   1995-05-08
23223   1995-05-08
32192   1995-05-08
43599   1995-05-08
56922   1995-05-08
9537    1995-05-08
1775    1995-05-08
98709   1995-05-08
52879   1995-05-08
7256    1995-05-08
Name: Open Date, dtype: datetime64[ns]

In [32]:
# Add datetime parameters for saledate
df_test["reveuneYear"] = df_test['Open Date'].dt.year
df_test["revenueMonth"] = df_test['Open Date'].dt.month
df_test["revenueDay"] = df_test['Open Date'].dt.day
df_test["revenueDayofweek"] = df_test['Open Date'].dt.dayofweek
df_test["revenueDayofyear"] = df_test['Open Date'].dt.dayofyear

# Drop original saledate
df_test.drop("Open Date", axis=1, inplace=True)

In [34]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 16457 to 9575
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                100000 non-null  int64  
 1   City              100000 non-null  object 
 2   City Group        100000 non-null  object 
 3   Type              100000 non-null  object 
 4   P1                100000 non-null  int64  
 5   P2                100000 non-null  float64
 6   P3                100000 non-null  float64
 7   P4                100000 non-null  float64
 8   P5                100000 non-null  int64  
 9   P6                100000 non-null  int64  
 10  P7                100000 non-null  int64  
 11  P8                100000 non-null  int64  
 12  P9                100000 non-null  int64  
 13  P10               100000 non-null  int64  
 14  P11               100000 non-null  int64  
 15  P12               100000 non-null  int64  
 16  P13               

In [35]:
for label, content in df_test.items():
    if pd.api.types.is_string_dtype(content):
        df_test[label] = content.astype("category").cat.as_ordered()

for label, content in df_test.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_test[label] = pd.Categorical(content).codes+1

In [36]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 16457 to 9575
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                100000 non-null  int64  
 1   City              100000 non-null  int8   
 2   City Group        100000 non-null  int8   
 3   Type              100000 non-null  int8   
 4   P1                100000 non-null  int64  
 5   P2                100000 non-null  float64
 6   P3                100000 non-null  float64
 7   P4                100000 non-null  float64
 8   P5                100000 non-null  int64  
 9   P6                100000 non-null  int64  
 10  P7                100000 non-null  int64  
 11  P8                100000 non-null  int64  
 12  P9                100000 non-null  int64  
 13  P10               100000 non-null  int64  
 14  P11               100000 non-null  int64  
 15  P12               100000 non-null  int64  
 16  P13               

In [40]:
# Splitting into training and testing

X_train = df_tmp.drop("revenue", axis=1)
y_train = df_tmp.revenue

X_train.head()

,Id,City,City Group,Type,P1,P2,P3,P4,P5,P6,...,P33,P34,P35,P36,P37,reveuneYear,revenueMonth,revenueDay,revenueDayofweek,revenueDayofyear
35,35,1,2,3,4,5.0,4.0,3.0,1,2,...,3,3,4,3,3,1996,5,8,2,129
122,122,32,1,3,12,7.5,6.0,7.5,2,8,...,6,18,3,16,6,1997,4,10,3,100
28,28,32,1,3,4,5.0,4.0,4.0,1,3,...,2,3,5,4,4,1998,1,28,2,28
112,112,32,1,3,4,5.0,5.0,4.0,1,5,...,2,2,1,3,3,1998,3,14,5,73
64,64,32,1,3,12,7.5,6.0,6.0,2,8,...,4,24,15,20,6,1998,7,13,0,194


In [41]:
y_train

35     3903884.0
122    4286645.0
28     4705945.0
112    3784230.0
64     2551252.0
         ...    
77     1904842.0
52     1763231.0
65     1756069.0
119    3780019.0
21     1149870.0
Name: revenue, Length: 137, dtype: float64

In [42]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

model.fit(X_train, y_train)

RandomForestRegressor()

In [43]:
y_preds = model.predict(df_test)

In [44]:
y_preds

array([5044551.27, 4089941.14, 5593192.26, ..., 5978681.72, 6894557.15,
       5780689.22])

## Moving Forward

1. Split into valid and training to tune parameters for df_tmp
2. EDA to see which features can be removed